# Mühle GUI

In [ ]:
import ipycanvas
import ipywidgets
import math

In [ ]:
# TODO: create schema to visualize following constants

BOARD_SIZE = 500
CANVAS_PADDING = 30
ROW_WIDTH = 60
PLAYER_PIECE_RADIUS = 12
DEFAULT_PIECE_RADIUS = 5

STASH_STONES_SPACING = 5

STASH_HEIGHT = ((PLAYER_PIECE_RADIUS * 2 ) * 2) + (STASH_STONES_SPACING) + (CANVAS_PADDING * 2)
STASH_WIDTH = ((PLAYER_PIECE_RADIUS * 2 ) * 9) + (STASH_STONES_SPACING * 8) + (CANVAS_PADDING * 2)

CANVAS_HEIGHT = BOARD_SIZE
CANVAS_WIDTH = BOARD_SIZE + STASH_WIDTH

STASH_STARTING_POINT_X = BOARD_SIZE
STASH_STARTING_POINT_Y = CANVAS_HEIGHT - STASH_HEIGHT

NO_PLAYER = ' '
PLAYER_1 = 'w'
PLAYER_2 = 'b'


# colors
BOARD_FOREGROUND_COLOR = '#191919'
BOARD_BACKGROUND_COLOR = '#ffffcb'
STASH_BACKGROUND_COLOR = '#dedede'
PLAYER_1_FILL_COLOR = '#E8E8E8'
PLAYER_1_STROKE_COLOR = '#191919'
PLAYER_2_FILL_COLOR = '#3D3D3D'
PLAYER_2_STROKE_COLOR = '#191919'

# shadows
SHADOW_COLOR = '#000000'
SHADOW_OFFSET_X = 2
SHADOW_OFFSET_Y = 2
SHADOW_BLUR = 2

# default shadows
SHADOW_COLOR_DEFAULT = 'rgba(0, 0, 0, 0)'
SHADOW_OFFSET_X_DEFAULT = 0
SHADOW_OFFSET_Y_DEFAULT = 0
SHADOW_BLUR_DEFAULT = 0


In [ ]:
# all available values for x and y, beginning left to right and top to bottom
av = [
    math.floor(CANVAS_PADDING),
    math.floor(CANVAS_PADDING + ROW_WIDTH),
    math.floor(CANVAS_PADDING + 2 * ROW_WIDTH),
    math.floor(BOARD_SIZE / 2),
    math.floor(BOARD_SIZE - (CANVAS_PADDING + 2 * ROW_WIDTH)),
    math.floor(BOARD_SIZE - (CANVAS_PADDING + ROW_WIDTH)),
    math.floor(BOARD_SIZE - CANVAS_PADDING)
]


In [ ]:
# All coordinates. The first tuple is the outer square, the second tuple is the middle square and the third tuple is the inner square. It always starts from top left and then clockwise.
coords = (
    (
        (av[0], av[0]),
        (av[3], av[0]),
        (av[6], av[0]),
        (av[6], av[3]),
        (av[6], av[6]),
        (av[3], av[6]),
        (av[0], av[6]),
        (av[0], av[3])
    ),
    (
        (av[1], av[1]),
        (av[3], av[1]),
        (av[5], av[1]),
        (av[5], av[3]),
        (av[5], av[5]),
        (av[3], av[5]),
        (av[1], av[5]),
        (av[1], av[3])
    ),
    (
        (av[2], av[2]),
        (av[3], av[2]),
        (av[4], av[2]),
        (av[4], av[3]),
        (av[4], av[4]),
        (av[3], av[4]),
        (av[2], av[4]),
        (av[2], av[3])
    )
) 

In [ ]:
# Function to draw a line on a canvas c, from start to end. Start and end are tuples with x and y values.
def draw_line(c, start, end):
    c.move_to(start[0], start[1])
    c.line_to(end[0], end[1])

In [ ]:
# Function to draw a circle on a canvas c with given coords, radius and color
def draw_circle(c, coords, radius, color, stroke_color = None, use_shadow = False):
    if use_shadow == True:
        c.shadow_color = SHADOW_COLOR
        c.shadow_offset_x = SHADOW_OFFSET_X
        c.shadow_offset_y = SHADOW_OFFSET_Y
        c.shadow_blur = SHADOW_BLUR
    c.fill_style = color
    c.fill_arc(coords[0], coords[1], radius, 0, 2 * math.pi)
    if use_shadow == True:
        c.shadow_color = SHADOW_COLOR_DEFAULT
        c.shadow_offset_x = SHADOW_OFFSET_X_DEFAULT
        c.shadow_offset_y = SHADOW_OFFSET_Y_DEFAULT
        c.shadow_blur = SHADOW_BLUR_DEFAULT
    if stroke_color is not None:
        c.stroke_color = stroke_color
        c.stroke_arc(coords[0], coords[1], radius, 0, 2 * math.pi)

In [ ]:
# Function to draw a (player) stone with given coords and for a given player (or the the default )
def draw_stone(c, coords, player):
    if player == PLAYER_1:
        draw_circle(c, coords, PLAYER_PIECE_RADIUS, PLAYER_1_FILL_COLOR, stroke_color = PLAYER_1_STROKE_COLOR, use_shadow = True)
        draw_circle(c, coords, math.floor(PLAYER_PIECE_RADIUS / 2), PLAYER_1_FILL_COLOR, stroke_color = PLAYER_1_STROKE_COLOR)
    elif player == PLAYER_2:
        draw_circle(c, coords, PLAYER_PIECE_RADIUS, PLAYER_2_FILL_COLOR, stroke_color = PLAYER_2_STROKE_COLOR, use_shadow = True)
        draw_circle(c, coords, math.floor(PLAYER_PIECE_RADIUS / 2), PLAYER_2_FILL_COLOR, stroke_color = PLAYER_2_STROKE_COLOR)
    else:
        draw_circle(c, coords, DEFAULT_PIECE_RADIUS, BOARD_FOREGROUND_COLOR)

In [ ]:
# Function to construct a square on a canvas c.
def construct_square(c, square):
    for i in range(4):
        start = i * 2
        end = (i * 2 + 2) if (i * 2 + 2 <= 6) else 0 
        draw_line(c, square[start], square[end])

In [ ]:
# Function to construct the cross lines on a canvas c
def construct_cross_lines(c, coords):
    for i in range(4):
        k = i * 2 + 1
        draw_line(c, coords[0][k], coords[2][k])

In [ ]:
# Initialize the canvas and draw the game board 
canvas = ipycanvas.MultiCanvas(2, width = CANVAS_WIDTH, height = CANVAS_HEIGHT)
canvas[0].fill_style = BOARD_BACKGROUND_COLOR
canvas[0].fill_rect(0, 0, BOARD_SIZE, BOARD_SIZE)

canvas[0].fill_style = STASH_BACKGROUND_COLOR
canvas[0].fill_rect(STASH_STARTING_POINT_X, STASH_STARTING_POINT_Y, CANVAS_WIDTH, CANVAS_HEIGHT)


canvas[0].stroke_style = BOARD_FOREGROUND_COLOR
canvas[0].begin_path()

for i in range(3):
    construct_square(canvas[0], coords[i])
    
    

construct_cross_lines(canvas[0], coords)

canvas[0].stroke()

canvas

In [ ]:
def update_gui(state):
    # TODO: ADD INTERACTIVITY
    canvas[1].clear()
    ((stash_p1, stash_p2), squares) = state
    
    # update pieces on the board
    for i in range(len(squares)):
        for j in range(len(squares[i])):
            draw_stone(canvas[1], coords[i][j], squares[i][j])

    # update pieces on the stash

    # player 1

    x = STASH_STARTING_POINT_X + PLAYER_PIECE_RADIUS
    y = STASH_STARTING_POINT_Y + CANVAS_PADDING + PLAYER_PIECE_RADIUS

    for i in range(stash_p1):
        x += 2 * PLAYER_PIECE_RADIUS + STASH_STONES_SPACING
        draw_stone(canvas[1], (x, y), PLAYER_1)

    # player 2

    x = STASH_STARTING_POINT_X + PLAYER_PIECE_RADIUS
    y = STASH_STARTING_POINT_Y + CANVAS_PADDING + 3 * PLAYER_PIECE_RADIUS + STASH_STONES_SPACING

    for i in range(stash_p2):
        x += 2 * PLAYER_PIECE_RADIUS + STASH_STONES_SPACING
        draw_stone(canvas[1], (x, y), PLAYER_2)
    

In [ ]:
# Test States
s1 = ((9, 9), (
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ')
))
s2 = ((0, 0), (
    ('w', 'w', ' ', 'b', 'w', 'b', 'w', 'w'),
    ('b', 'w', ' ', 'b', 'w', ' ', ' ', 'w'),
    ('b', 'b', 'b', 'b', ' ', ' ', 'b', 'w'),
))
s3 = ((7, 7), (
    ('w', 'w', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', 'b', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', 'b', ' ', ' ')
))
s4 = ((2, 8), (
    ('w', 'w', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', 'b', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', 'b', ' ', ' ')
))

In [ ]:
update_gui(s1)
canvas

In [ ]:
update_gui(s2)

In [ ]:
update_gui(s3)

In [ ]:
update_gui(s4)